<a href="https://colab.research.google.com/github/thiagosantos346/PNL_MODELS/blob/master/pnl_multlabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento de linguagem natural : Atributos multi-rotulado.

## Importação dos dados:

In [0]:
import pandas as pd

In [0]:
data_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/stackoverflow/stackoverflow_perguntas.csv')

In [227]:
data_set.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


## Limpando os dados:

Aparentemente, temos uma classificação com multiplos rotulos, pois vendo a coluna tags, notamos que na linha 4, existe dois rotulos para a pergunta...

Agoras é interessante saber também o tamanho do nosso problema...
 - Vamos ver o tamanho total do conjunto de dados(dataset)
 - Depois a quantidade de tags únicas...


In [228]:
dataset_size = len(data_set)
message_size =  'Número de linhas no dataset:{}.'.format(dataset_size)
print (message_size)

Número de linhas no dataset:5408.


In [229]:
tag_list = data_set.Tags
size = len(tag_list.unique())
tag_list = tag_list.unique()

message_size =  'Número de valores únicos:{size}.\n Valores :\n {values}'
message_size =  message_size.format(size=size, values=tag_list)

print(message_size)


Número de valores únicos:37.
 Valores :
 ['node.js' 'jquery' 'html' 'html angular ' 'html ' 'angular' 'angular '
 'jquery html  ' 'jquery ' 'jquery html' 'jquery html ' 'html angular'
 'angular node.js ' 'html  ' 'jquery html angular' 'node.js '
 'html jquery' 'html jquery ' 'jquery angular  ' 'html node.js' 'jquery  '
 'angular node.js' 'jquery angular' 'html node.js ' 'jquery node.js '
 'angular  ' 'jquery angular ' 'jquery html angular ' 'node.js html '
 ' node.js' 'node.js html' 'html angular  ' 'jquery node.js'
 'angular html' 'html angular  node.js' 'jquery html node.js'
 'html angular node.js']


Feito isso podemos notar alguns problemas que devemos tratar:
 - A ordem das palavras estão gerando novos valores únicos;
 - Erros e ruidos tamabém produzem novos valores únicos;
 

In [230]:
def get_clean_tag_list(tags_list):
  cleaned_tag_list = list()
  for tags in tags_list:
    tags_splited = tags.split()
    for tag in tags_splited:
      if tag not in cleaned_tag_list:
        cleaned_tag_list.append(tag)
  return cleaned_tag_list

cleaned_tag_list = get_clean_tag_list(tag_list)
print(cleaned_tag_list)


['node.js', 'jquery', 'html', 'angular']


In [231]:
print(data_set['Tags'])

0             node.js
1             node.js
2              jquery
3                html
4       html angular 
            ...      
5403     jquery html 
5404            html 
5405      jquery html
5406             html
5407      jquery html
Name: Tags, Length: 5408, dtype: object


In [0]:
def count_label_in_row(label_list, data_set, column_name='Tags'): 
  for tag in label_list:
    count_list = list()
    for column in data_set[column_name]:
      if tag in column:
        value = 1
      else:
        value = 0
      count_list.append(value)
    
    data_set[tag] = count_list

In [233]:
count_label_in_row(cleaned_tag_list, data_set)
data_set.head(100)

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1
...,...,...,...,...,...,...
95,"Tenho um projeto desenvolvido em angular 1, es...",angular,0,0,0,1
96,Tenho esse html: Gostaria que quando o usuá...,jquery html,0,1,1,0
97,"Ao incluir um item dinamicamente na CODE , eu ...",jquery,0,1,0,0
98,"Gostaria de saber a maneira correta, seguindo ...",jquery,0,1,0,0


Agora que criamos uma coluna para cada tag unica, e marcamos o número de ocorrências dessa tag na linha podemos prosseguir para dividir as massa de dados para treino e teste.

## Separação do conjunto de dados para teste e treino:

### Zipar as colunas
1. Temos uma pequena diferença da divisão feita nos modelos mais simples:
 - Temos 4 colunas para representar apenas uma entidade;
 - Nosso divisor precisa de um atributo que represente as 4 colunas em apenas uma.

  Sendo assim ao dividir os dados de teste e treino poderiamos ficar com dados 
  não proporcionais a realidade, isso é uma divisão de dados que não contivesse uma das tags que queremos treinar o nosos modelo, a ideia é dividir os dados de maneira que não falte entidades na massa de dados para treino, evitando assim testar um valor ao qual o modelo não foi treinado.

2. Solução:
 - Criar apenas uma coluna contendo o crusamento dos valores da lita.
 - Para isso vamos usar uma função de zipagens dos dados a funcção `zip()`.
 - Assin contruindo uma coluna de tupulas das cominações de de cada rotulo que queremos prover.


In [234]:
l1 = data_set[cleaned_tag_list[0]]
l2 = data_set[cleaned_tag_list[1]]
l3 = data_set[cleaned_tag_list[2]]
l4 = data_set[cleaned_tag_list[3]]
ziped_tag_list = list(zip( l1, l2, l3, l4))

data_set['all_tagas_tupules'] =  ziped_tag_list
data_set.head()

,Perguntas,Tags,node.js,jquery,html,angular,all_tagas_tupules
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


### Seprar os dados:

In [0]:
from sklearn.model_selection import train_test_split
import random

fit_input_dataset, test_input_dataset, fit_output_dataset, test_output_dataset = train_test_split(
    data_set['Perguntas'],
    data_set['all_tagas_tupules'],
    test_size=0.2,
    random_state=random.randint(1, 1000)
)

#### Vetorização do input de treino:

##### TF-IDF

###### Configurar o vetorizador :
 - Vamos definir o número maximo de linhas no vetor com `max_features`.
 - Vamos retirar as palavras que se repetem muinto com `max_df`.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
MAX_FEATURES = 5000
MAX_DISTANCE_FREQUENCIE = 0.85
vectorizer = TfidfVectorizer(max_features=MAX_FEATURES, max_df=MAX_DISTANCE_FREQUENCIE)

###### Vamos criar o vetor de frequência de palavras no texto:

In [0]:
vectorizer.fit(data_set['Perguntas'])
fit_input_dataset_tfidf = vectorizer.transform(fit_input_dataset)
test_input_dataset_tfidf = vectorizer.transform(test_input_dataset)

In [238]:
print(fit_input_dataset_tfidf.shape)
print(test_input_dataset_tfidf.shape)

(4326, 5000)
(1082, 5000)


Como vemos foram geradas 5000 linhas, e combinadas com a frequência da proporção de pareto 80/20, para treino e teste.

## Treinando o modelo:

**Relevância Binária**

Essa estratégia se base em uma catégorização binaria simples, porem aplicada a cada uma das tupulas que queremos testar.

### Classificador:
Usáremos o algoritmo **OneVsRestClassifier**, que tem como **Requesitos**:
  - Um **Estimador**, usaremos a **Regreção lógistica**.

### Regreção Lógistica :

In [0]:
from sklearn.linear_model import LogisticRegression

instance_LogisticRegression = LogisticRegression(solver='lbfgs')

### OneVsOneClassifier

In [0]:
from sklearn.multiclass import OneVsRestClassifier

instance_OneVsRestClassifier = OneVsRestClassifier(instance_LogisticRegression)

### Tranformação pre-treino:

O nosso classificador recebe como entrada um arranjo binário, porem os nosso dados de treinos são do tipo serie, para resolver isso precisamos converter esses dados em um arranjo binário ou uma matriz esparsa.

In [241]:
type(fit_output_dataset)

pandas.core.series.Series

Além dessa transformação, precisamos fazer as tupulas se tornarem listas, por isso o list em volta dos dados de treino e test de saída.

In [0]:
import numpy as np

array_fit_output_dataset = np.asarray(list(fit_output_dataset))
array_test_output_dataset = np.asarray(list(test_output_dataset))

In [243]:
len(array_test_output_dataset)

1082

### Executando o Treinamento :

Agora basta treinar o modelo com os TF-IDF e os Arrays de saída, criados anteriormente:

In [244]:
instance_OneVsRestClassifier.fit(fit_input_dataset_tfidf, array_fit_output_dataset)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

## Avaliando o modelo

### Acurácia : 

A acurácia não é o melhor meio de avaliar o nosso modelo, pois é baseda em `Exact Match` que seria uma combinação exata das 4 colunas que queremos 
predizer.

In [245]:
score = instance_OneVsRestClassifier.score(test_input_dataset_tfidf, array_test_output_dataset)
mensage_model_acuracy = 'Acurácia atribuida : {0: .2f}%'.format(score*100)
print(mensage_model_acuracy)

Acurácia atribuida :  38.72%


Só para termos uma noção, vamos ver quantas combinações possiveis existem no nosso resultado.

In [246]:
choices = len(data_set.all_tagas_tupules.unique())
print(choices)

13


O nosso total é:

In [247]:
options = len(data_set.all_tagas_tupules)
print(options)

5408


Com uma escolha aleátoria termismos a seguinte chances de acertar: 

In [248]:
mensage_random_choice  = 'Acurácia atribuida : {0: .2f}%'.format((choices/options) * 100)
print(mensage_random_choice ) 

Acurácia atribuida :  0.24%


### Hamming Loss

  Esse metodo de avaliação trabalha com distância de elementos por linha, em outras palavras se tenho um **total de 4 atributos** para predizer e foram **obitidos 3 elementos corretos** a distância entre o total e o números de acerto é de **`` 4 - 3 = 1 ``**.
  Quanto mais proxima de zero for essa distância melhor será os nossos resultados.

In [0]:
from sklearn.metrics import hamming_loss

prediction_oneVsRest =  instance_OneVsRestClassifier.predict(test_input_dataset_tfidf)
hamming_loss_oneVsRest = hamming_loss(array_test_output_dataset, prediction_oneVsRest)


In [259]:
mensage_hamming_loss_distance  = 'Distância entre real e previsto(Hamming Loss): {0: .2f}.'.format(hamming_loss_oneVsRest)
print(mensage_hamming_loss_distance ) 

Distância entre real e previsto(Hamming Loss):  0.19.


Podemos ver também a correlação entre cada uma das variaveis, já que nesse modelo esstas foram tratadas de forma indepêndentes.

In [260]:
data_set.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


Notamos que nenhuma das variáveis da matriz veio com 0, existe pelo menos uma correlação inversa, o que levanta a duvida, seria possivel melhorar o desempenho desse modelo, levando enconta essas correlações?